## Libaries

In [1]:
# Main libraries installation
!pip install geopandas requests contextily geopy faker > nul

In [2]:
# Import necessary libraries
import random
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import geopandas as gpd      # For working with geospatial data
import pandas as pd          # For data manipulation and analysis
import os                    # For operating system-related functionality
import io                    # For input/output operations
import contextily as cx      # For basemaps and context tiles
import matplotlib.pyplot as plt  # For creating visualizations
from rasterio.crs import CRS  # For handling coordinate reference systems
from math import sin, asin, fmod, pi
import numpy as np
import plotly.express as px  # Library for creating interactive visualizations
from faker import Faker
import string
import plotly.io as pio  # Library for handling Plotly I/O
from IPython.display import Image  # Library for displaying images in IPython environment
pd.options.display.max_columns=None

## Data Ingestion

In [3]:
def generar_user_code():
    caracteres = list(string.ascii_letters + string.digits)
    random.shuffle(caracteres)
    return ''.join(caracteres[:10])

def generar_puntos_ciudad_mexico(cantidad):
    latitud_min, latitud_max = 19.0, 19.6
    longitud_min, longitud_max = -99.4, -98.9
    puntos = []

    for _ in range(cantidad):
        # Generar coordenadas aleatorias
        latitud = round(random.uniform(latitud_min, latitud_max), 5)
        longitud = round(random.uniform(longitud_min, longitud_max), 5)
        
        # Generar User Code
        user_code = generar_user_code()
        
        puntos.append((latitud, longitud, user_code))

    return puntos

# Generar 30 puntos en la Ciudad de México
puntos_ciudad_mexico = generar_puntos_ciudad_mexico(500)

# Crear un DataFrame con los puntos y User Code
columnas = ['Latitude', 'Longitude', 'User Code']
df = pd.DataFrame(puntos_ciudad_mexico, columns=columnas)

df.head()

,Latitude,Longitude,User Code
0,19.27934,-98.96953,tGbJy9OAmU
1,19.59892,-99.35482,DIBhWRZEQc
2,19.17519,-99.29773,YuQfvn7sag
3,19.48921,-99.09211,NvR2cVhO15
4,19.21433,-98.98875,FtNTDVApyQ


## Data exploring

In [4]:
# Convertimos latitud y longitud de coordenadas cartesianas a coordenadas geográficas
df['lat'] = df['Latitude'].apply(lambda lat: asin(sin((lat/180.0)*pi)) * (180.0/pi))
df['lon'] = df['Longitude'].apply(lambda lon: fmod(lon - 180.0, 360.0) + 180.0)

In [5]:
# Unimos las coordenadas convertidas y las convertimos en una dirección humanamente legible
locator = Nominatim(user_agent='my_app/0,1', timeout=76000)
df['lat_lon'] = df['lat'].astype(str) + ',' + df['lon'].astype(str)
df['location'] = df['lat_lon'].apply(lambda lat_lon:  locator.reverse(lat_lon, language='en'))

In [14]:
df.head()

,Latitude,Longitude,User Code,lat,lon,lat_lon,calle,numero_casa,codigo_postal,amenity,vecindario,suburbio,estructura,vias_ferreas,autopista,aldea,villa,poblado,vivienda_aislada,asentamiento,zona_residencial,municipio,ciudad,distrito,provincia,region,condado,estado,distrito_estatal,pais,codigo_pais,pais_iso,subdistrito_iso,info_turistica,info_edilicia,info_oficina,info_tienda,info_industrial,info_historica
0,19.27934,-98.96953,tGbJy9OAmU,19.27934,-98.96953,"19.27934,-98.96953000000002",Calle Río Ameca,NaN,56613,NaN,Ampliación San Miguel Tláhuac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Valle de Chalco Solidaridad,NaN,NaN,NaN,NaN,State of Mexico,NaN,Mexico,mx,MX-MEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19.59892,-99.35482,DIBhWRZEQc,19.59892,-99.35482,"19.59892,-99.35482000000002",Las Palomas,NaN,54435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nicolás Romero,NaN,NaN,NaN,Nicolás Romero,NaN,NaN,Mexico,mx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19.17519,-99.29773,YuQfvn7sag,19.17519,-99.29773,"19.17519,-99.29773",NaN,NaN,52690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Las Cocinas,Xalatlaco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mexico,mx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19.48921,-99.09211,NvR2cVhO15,19.48921,-99.09211,"19.48921,-99.09210999999999",Calle 300-A,NaN,07420,NaN,Unidad Habitacional El Coyol,Gustavo A. Madero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mexico City,NaN,NaN,NaN,NaN,NaN,NaN,Mexico,mx,MX-CMX,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19.21433,-98.98875,FtNTDVApyQ,19.21433,-98.98875,"19.214329999999997,-98.98874999999998",Cerrada Callejón de los Mecales,NaN,12100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Milpa Alta,NaN,San Antonio Tecómitl,NaN,NaN,Mexico City,NaN,NaN,NaN,NaN,NaN,NaN,Mexico,mx,MX-CMX,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df['dirección'] = df['location'].apply(lambda loc: loc.raw['address'])

KeyError: 'location'

In [16]:
# Utilizamos el json con direcciones y convertimos cada atributo en columna
df['calle'] = df['dirección'].apply(lambda addr: addr.get('road', np.nan))
df['numero_casa'] = df['dirección'].apply(lambda addr: addr.get('house_number', np.nan))
df['codigo_postal'] = df['dirección'].apply(lambda addr: addr.get('postcode', np.nan))
df['amenity'] = df['dirección'].apply(lambda addr: addr.get('amenity', np.nan))
df['vecindario'] = df['dirección'].apply(lambda addr: addr.get('neighbourhood', np.nan))
df['suburbio'] = df['dirección'].apply(lambda addr: addr.get('suburb', np.nan))
df['estructura'] = df['dirección'].apply(lambda addr: addr.get('man_made', np.nan))
df['vias_ferreas'] = df['dirección'].apply(lambda addr: addr.get('railway', np.nan))
df['autopista'] = df['dirección'].apply(lambda addr: addr.get('highway', np.nan))
df['aldea'] = df['dirección'].apply(lambda addr: addr.get('hamlet', np.nan))
df['villa'] = df['dirección'].apply(lambda addr: addr.get('village', np.nan))
df['poblado'] = df['dirección'].apply(lambda addr: addr.get('town', np.nan))
df['vivienda_aislada'] = df['dirección'].apply(lambda addr: addr.get('isolated_dwelling', np.nan))
df['asentamiento'] = df['dirección'].apply(lambda addr: addr.get('quarter', np.nan))
df['zona_residencial'] = df['dirección'].apply(lambda addr: addr.get('residential', np.nan))
df['municipio'] = df['dirección'].apply(lambda addr: addr.get('municipality', np.nan))
df['ciudad'] = df['dirección'].apply(lambda addr: addr.get('city', np.nan))
df['distrito'] = df['dirección'].apply(lambda addr: addr.get('city_district', np.nan))
df['provincia'] = df['dirección'].apply(lambda addr: addr.get('province', np.nan))
df['region'] = df['dirección'].apply(lambda addr: addr.get('region', np.nan))
df['condado'] = df['dirección'].apply(lambda addr: addr.get('county', np.nan))
df['estado'] = df['dirección'].apply(lambda addr: addr.get('state', np.nan))
df['distrito_estatal'] = df['dirección'].apply(lambda addr: addr.get('state_district', np.nan))
df['pais'] = df['dirección'].apply(lambda addr: addr.get('country', np.nan))
df['codigo_pais'] = df['dirección'].apply(lambda addr: addr.get('country_code', np.nan))
df['pais_iso'] = df['dirección'].apply(lambda addr: addr.get('ISO3166-2-lvl4', np.nan))
df['subdistrito_iso'] = df['dirección'].apply(lambda addr: addr.get('ISO3166-2-lvl15', np.nan))
df['info_turistica'] = df['dirección'].apply(lambda addr: addr.get('tourism', np.nan))
df['info_edilicia'] = df['dirección'].apply(lambda addr: addr.get('building', np.nan))
df['info_oficina'] = df['dirección'].apply(lambda addr: addr.get('office', np.nan))
df['info_tienda'] = df['dirección'].apply(lambda addr: addr.get('shop', np.nan))
df['info_industrial'] = df['dirección'].apply(lambda addr: addr.get('industrial', np.nan))
df['info_historica'] = df['dirección'].apply(lambda addr: addr.get('historic', np.nan))

KeyError: 'dirección'

In [9]:
# Dropeamos las columnas tipo json para poder exportar y viausalizar el df
df=df.drop(['location', 'dirección'], axis=1)

In [10]:
# visualizamos las columnas generadas
df.head(10)

,Latitude,Longitude,User Code,lat,lon,lat_lon,calle,numero_casa,codigo_postal,amenity,vecindario,suburbio,estructura,vias_ferreas,autopista,aldea,villa,poblado,vivienda_aislada,asentamiento,zona_residencial,municipio,ciudad,distrito,provincia,region,condado,estado,distrito_estatal,pais,codigo_pais,pais_iso,subdistrito_iso,info_turistica,info_edilicia,info_oficina,info_tienda,info_industrial,info_historica
0,19.27934,-98.96953,tGbJy9OAmU,19.27934,-98.96953,"19.27934,-98.96953000000002",Calle Río Ameca,NaN,56613,NaN,Ampliación San Miguel Tláhuac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Valle de Chalco Solidaridad,NaN,NaN,NaN,NaN,State of Mexico,NaN,Mexico,mx,MX-MEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19.59892,-99.35482,DIBhWRZEQc,19.59892,-99.35482,"19.59892,-99.35482000000002",Las Palomas,NaN,54435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nicolás Romero,NaN,NaN,NaN,Nicolás Romero,NaN,NaN,Mexico,mx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19.17519,-99.29773,YuQfvn7sag,19.17519,-99.29773,"19.17519,-99.29773",NaN,NaN,52690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Las Cocinas,Xalatlaco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mexico,mx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19.48921,-99.09211,NvR2cVhO15,19.48921,-99.09211,"19.48921,-99.09210999999999",Calle 300-A,NaN,07420,NaN,Unidad Habitacional El Coyol,Gustavo A. Madero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mexico City,NaN,NaN,NaN,NaN,NaN,NaN,Mexico,mx,MX-CMX,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19.21433,-98.98875,FtNTDVApyQ,19.21433,-98.98875,"19.214329999999997,-98.98874999999998",Cerrada Callejón de los Mecales,NaN,12100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Milpa Alta,NaN,San Antonio Tecómitl,NaN,NaN,Mexico City,NaN,NaN,NaN,NaN,NaN,NaN,Mexico,mx,MX-CMX,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,19.41712,-99.35761,dhGZFXVfcj,19.41712,-99.35761,"19.41712,-99.35761000000002",Autopista Toluca - Naucalpan,NaN,53650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,La Cumbre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Huixquilucan,NaN,NaN,Mexico,mx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,19.48283,-99.38796,W5EbL8NOUk,19.48283,-99.38796,"19.48283,-99.38796000000002",NaN,NaN,54570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Ayucan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jilotzingo,NaN,NaN,Mexico,mx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,19.24840,-99.05564,qOEoLbkrAt,19.24840,-99.05564,"19.2484,-99.05563999999998",Cerrada de Acueducto,NaN,16605,NaN,Colonia Los Reyes,Xochimilco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mexico City,NaN,NaN,NaN,NaN,NaN,NaN,Mexico,mx,MX-CMX,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,19.01135,-99.39228,rX28P6RDxh,19.01135,-99.39228,"19.01135,-99.39228000000003",Calle Morelos,NaN,NaN,NaN,Colonia Doctor Gustavo Baz,NaN,NaN,NaN,NaN,NaN,NaN,Santa María Nativitas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ocuilan,NaN,NaN,Mexico,mx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,19.52053,-98.91113,NTeRmAC7Ki,19.52053,-98.91113,"19.52053,-98.91113000000001",Cerrada Primera Cerrada De Leona Vicario,NaN,56203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tocuila,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Texcoco,NaN,NaN,Mexico,mx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Calcula el porcentaje de completitud para cada columna
num_rows = len(df)
for col in df.columns:
    num_non_null = df[col].notnull().sum()
    completeness_pct = num_non_null / num_rows * 100
    print(f"Completitud de {col}: {completeness_pct:.2f}%")

Completitud de Latitude: 100.00%
Completitud de Longitude: 100.00%
Completitud de User Code: 100.00%
Completitud de lat: 100.00%
Completitud de lon: 100.00%
Completitud de lat_lon: 100.00%
Completitud de calle: 55.00%
Completitud de numero_casa: 4.40%
Completitud de codigo_postal: 91.80%
Completitud de amenity: 1.40%
Completitud de vecindario: 30.40%
Completitud de suburbio: 29.20%
Completitud de estructura: 0.20%
Completitud de vias_ferreas: 0.00%
Completitud de autopista: 0.00%
Completitud de aldea: 1.20%
Completitud de villa: 22.40%
Completitud de poblado: 16.80%
Completitud de vivienda_aislada: 0.00%
Completitud de asentamiento: 2.60%
Completitud de zona_residencial: 2.60%
Completitud de municipio: 0.00%
Completitud de ciudad: 56.60%
Completitud de distrito: 0.80%
Completitud de provincia: 0.00%
Completitud de region: 0.00%
Completitud de condado: 47.00%
Completitud de estado: 21.20%
Completitud de distrito_estatal: 0.00%
Completitud de pais: 100.00%
Completitud de codigo_pais: 100

## Data Visualization

In [12]:
# Create a map using Plotly Express
fig = px.scatter_mapbox(df, 
                        lat=df.lat, 
                        lon=df.lon,
                        color='User Code',
                        color_continuous_scale='blues',
                        mapbox_style='dark',
                        zoom=12)

# Display the map
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [13]:
# Assuming 'fig' is your Plotly figure
pio.write_html(fig, 'data/reverse.html')